In [19]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/2d/d2/6473460b1aad0944e8f0d7d618723f4b9d60485554eb1d42c5b193e716ec/pandas-1.2.2-cp37-cp37m-macosx_10_9_x86_64.whl
  Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3


In [20]:
import sklearn
import numpy as np
import pandas as pd
import mlrose
import mlrose_hiive
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import f1_score
from functools import partial
from IPython.display import display
from sklearn.model_selection import learning_curve, cross_val_score, train_test_split, cross_val_predict, StratifiedKFold, cross_validate, GridSearchCV, ShuffleSplit
from sklearn.metrics import confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score, average_precision_score, roc_curve, accuracy_score, make_scorer, classification_report
from sklearn import preprocessing
pd.options.display.max_columns = None
matplotlib.rc('figure', figsize=[10,5])

In [21]:
df_churn = pd.read_csv('data/Telecom_customer_churn.csv', index_col = False)
print(df_churn.dtypes)
print(f'unique Value of our Target_class:  {df_churn.Churn.unique()}')

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object
unique Value of our Target_class:  ['No' 'Yes']


# Creating categorical values nto one hot encoding and dropping null values 

In [22]:
#Removing missing values 
df_churn.dropna(inplace = True)
#Remove customer IDs from the data set
df2 = df_churn.iloc[:,1:]
#Convertin the predictor variable in a binary numeric variable
df2['Churn'].replace(to_replace='Yes', value=1, inplace=True)
df2['Churn'].replace(to_replace='No',  value=0, inplace=True)
# normalize features for better performance 
#Let's convert all the categorical variables into dummy variables
df_dummies = pd.get_dummies(df2)
df_dummies.head()

,SeniorCitizen,tenure,MonthlyCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TotalCharges_,TotalCharges_100.2,TotalCharges_100.25,TotalCharges_100.35,TotalCharges_100.4,TotalCharges_100.8,TotalCharges_100.9,TotalCharges_1001.2,TotalCharges_1001.5,TotalCharges_1003.05,TotalCharges_1004.35,TotalCharges_1004.5,TotalCharges_1004.75,TotalCharges_1005.7,TotalCharges_1006.9,TotalCharges_1007.8,TotalCharges_1007.9,TotalCharges_1008.55,TotalCharges_1008.7,TotalCharges_1009.25,TotalCharges_101.1,TotalCharges_101.45,TotalCharges_101.65,TotalCharges_101.9,TotalCharges_1010,TotalCharges_1011.05,TotalCharges_1011.5,TotalCharges_1011.8,TotalCharges_1012.4,TotalCharges_1013.05,TotalCharges_1013.2,TotalCharges_1013.35,TotalCharges_1013.6,TotalCharges_1014.25,TotalCharges_1016.7,TotalCharges_1017.35,TotalCharges_102.45,TotalCharges_102.5,TotalCharges_102.75,TotalCharges_1020.2,TotalCharges_1020.75,TotalCharges_1021.55,TotalCharges_1021.75,TotalCharges_1021.8,TotalCharges_1022.5,TotalCharges_1022.6,TotalCharges_1022.95,TotalCharges_1023.75,TotalCharges_1023.85,TotalCharges_1023.9,TotalCharges_1023.95,TotalCharges_1024,TotalCharges_1024.65,TotalCharges_1024.7,TotalCharges_1025.05,TotalCharges_1025.15,TotalCharges_1025.95,TotalCharges_1026.35,TotalCharges_1027.25,TotalCharges_1028.75,TotalCharges_1028.9,TotalCharges_1029.35,TotalCharges_1029.75,TotalCharges_1029.8,TotalCharges_103.7,TotalCharges_1031.1,TotalCharges_1031.4,TotalCharges_1031.7,TotalCharges_1032,TotalCharges_1032.05,TotalCharges_1033,TotalCharges_1033.9,TotalCharges_1033.95,TotalCharges_1035.5,TotalCharges_1035.7,TotalCharges_1036,TotalCharges_1036.75,TotalCharges_1037.75,TotalCharges_1039.45,TotalCharges_104.2,TotalCharges_104.3,TotalCharges_1041.8,TotalCharges_1042.65,TotalCharges_1043.3,TotalCharges_1043.35,TotalCharges_1043.4,TotalCharges_1043.8,TotalCharges_1045.25,TotalCharges_1046.1,TotalCharges_1046.2,TotalCharges_1046.5,TotalCharges_1047.7,TotalCharges_1048.45,TotalCharges_1048.85,TotalCharges_1049.05,TotalCharges_1049.6,TotalCharges_105.5,TotalCharges_105.6,TotalCharges_1050.5,TotalCharges_1051.05,TotalCharges_1051.9,TotalCharges_1052.35,TotalCharges_1052.4,TotalCharges_1054.6,TotalCharges_1054.75,TotalCharges_1054.8,TotalCharges_1055.9,TotalCharges_1056.95,TotalCharges_1057,TotalCharges_1057.55,TotalCharges_1057.85,TotalCharges_1058.1,TotalCharges_1058.25,TotalCharges_1058.6,TotalCharges_1059.55,TotalCharges_106.2,TotalCharges_106.55,TotalCharges_106.8,TotalCharges_106.85,TotalCharges_106.9,TotalCharges_1060.2,TotalCharges_1060.6,TotalCharges_1061.6,TotalCharges_1062.1,TotalCharges_1064.65,TotalCharges_1064.95,TotalCharges_1066.15,TotalCharges_1066.9,TotalCharges_1067.05,TotalCharges_1067.15,TotalCharges_1067.65,TotalCharges_1068.15,TotalCharges_1068.85,TotalCharges_107.05,TotalCharges_107.1,TotalCharges_107.25,TotalCharges_107.6,TotalCharges_1070.15,TotalCharges_1070.25,TotalCharges_1070.5,TotalCharges_1070.7,TotalCharges_1071.4,TotalCharges_1071.6,TotalCharges_1072,TotalCharges_1072.6,TotalCharges_1073.3,TotalCharges_1074.3,TotalCharges_1074.65,TotalCharges_1077.05,TotalCharges_1077.5,TotalCharges_1077.95,TotalCharges_1078.

In [23]:
predictors_list = ['tenure','InternetService_Fiber optic', 'InternetService_No', 'PaymentMethod_Electronic check',
       'OnlineSecurity_No', 'OnlineBackup_No', 'OnlineBackup_No internet service',  'StreamingTV_No internet service',
       'OnlineSecurity_No internet service','DeviceProtection_No','StreamingMovies_No internet service',
       'DeviceProtection_No internet service','TechSupport_No', 'TechSupport_No internet service','Contract_Two year','Contract_Month-to-month']


In [24]:
# min_max_scaler 

target = df_dummies['Churn']
X = df_dummies[predictors_list] # convert to numpy array
print(X.shape)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)

(7043, 16)


In [25]:
print(x_scaled.shape)
df_churn_scaled = pd.DataFrame(x_scaled, columns=predictors_list)
df_churn_scaled['Churn'] = target
df_churn_scaled.head()

(7043, 16)


,tenure,InternetService_Fiber optic,InternetService_No,PaymentMethod_Electronic check,OnlineSecurity_No,OnlineBackup_No,OnlineBackup_No internet service,StreamingTV_No internet service,OnlineSecurity_No internet service,DeviceProtection_No,StreamingMovies_No internet service,DeviceProtection_No internet service,TechSupport_No,TechSupport_No internet service,Contract_Two year,Contract_Month-to-month,Churn
0,0.013889,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0
1,0.472222,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,0.027778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1
3,0.625000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.027778,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1


# helper functions 

In [26]:
def split_data(df,feature_ls, y_target, training: bool = False) -> None:
        """
        Split the data into train, test for model building
        
        returns the Test and train data
        """
        if training:
            df = df.sample(frac=1)
            X_train, X_test, y_train,y_test = train_test_split(df[feature_ls], df[y_target], test_size=0.2, random_state=123) 
            print(X_train.shape,X_test.shape)
            #changing object datatype to int 
            return X_train, X_test, y_train,y_test
        
def get_and_save_metrics(true_labels, pred_labels, y_scores):
    """
    Using Sklearn Metrics, returns dataframe of all the relevant evaluation metrics
    for binary classification. 
    """
    
    accuracy = accuracy_score(true_labels, pred_labels)
    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels).ravel()
    precision_1 = precision_score(true_labels, pred_labels, pos_label=1)
    precision_0 = precision_score(true_labels, pred_labels, pos_label=0)
    recall_1 = recall_score(true_labels, pred_labels, pos_label=1)
    recall_0 = recall_score(true_labels, pred_labels, pos_label=0)
    f1_score_1 = f1_score(true_labels, pred_labels, pos_label=1)

    f1_score_0 = f1_score(true_labels, pred_labels, pos_label=0)
    auc_score = roc_auc_score(true_labels, y_scores)
    metrics = [[accuracy, tn, tp, fn, fp, precision_1, precision_0, recall_1, recall_0, f1_score_1, f1_score_0, auc_score]]
    metrics_df = pd.DataFrame(metrics)
    metrics_df.columns = ['accuracy', 'true_negative', 'true_positive', 'false_negative', 'false_positive',
                          'precision_class_1', 'precision_class_0', 'recall_class_1', 'recall_class_0',
                          'f1_score_class_1', 'f1_score_class_0', 'roc_auc_score']
    return metrics_df


def learning_curve(estimator, X, y, title = "Learning curve"):
    """
    Plotting learning curve by running 3 fold cross validation
    we will be using f-1 score as we have seen there is class imbalance 
    """
    
    f1_train_mean = []
    f1_train_std = []  
    cv_mean = []
    cv_std = []  
    fit_mean = []
    fit_std = []  
    pred_mean = []
    pred_std = []  

    data_size_list = (np.linspace(.05, 1.0, 20) * len(y)).astype('int')

    for len_data in data_size_list:
        index = np.random.randint(X.shape[0], size=len_data)
        sampl_train_data = X.iloc[index, :]
        sampl_train_label = y.iloc[index]
        scores = cross_validate(estimator, sampl_train_data, sampl_train_label, cv=3, scoring='f1', return_train_score=True)

        f1_train_mean.append(np.mean(scores['train_score']))
        f1_train_std.append(np.std(scores['train_score']))
        cv_mean.append(np.mean(scores['test_score']))
        cv_std.append(np.std(scores['test_score']))
        fit_mean.append(np.mean(scores['fit_time']))
        fit_std.append(np.std(scores['fit_time']))
        pred_mean.append(np.mean(scores['score_time']))
        pred_std.append(np.std(scores['score_time']))

    f1_train_mean = np.array(f1_train_mean);
    f1_train_std = np.array(f1_train_std)
    cv_mean = np.array(cv_mean);
    cv_std = np.array(cv_std)
    fit_mean = np.array(fit_mean);
    fit_std = np.array(fit_std)
    pred_mean = np.array(pred_mean);
    pred_std = np.array(pred_std)
    
    
    # plotting Learning curve 
    plt.figure()
    plt.title("Learning curve : " + title)
    plt.xlabel("Training size")
    plt.ylabel("F1 Score")
    plt.fill_between(data_size_list, f1_train_mean - 2 * f1_train_std, f1_train_mean + 2 * f1_train_std, alpha=0.1)
    plt.fill_between(data_size_list, cv_mean - 2 * cv_std, cv_mean + 2 * cv_std, alpha=0.1)
    plt.plot(data_size_list, f1_train_mean, 'o-', label="Training Score")
    plt.plot(data_size_list, cv_mean, 'o-',label="Cross-Validation Score")
    plt.legend(loc="best")
    plt.show()
    
    # plotting time 
    plt.figure()
    plt.title("Time taken by classifier: " + title)
    plt.xlabel("Training size")
    plt.ylabel("Time  in (s)")
    plt.fill_between(data_size_list, fit_mean - 2 * fit_std, fit_mean + 2 * fit_std, alpha=0.1)
    plt.fill_between(data_size_list, pred_mean - 2 * pred_std, pred_mean + 2 * pred_std, alpha=0.1)
    plt.plot(data_size_list, fit_mean, 'o-',  label="Training Time  in (s)")
    plt.plot(data_size_list, pred_std, 'o-', label="Prediction Time in (s)")
    plt.legend(loc="best")
    plt.show()


    # plot_times(data_size_list, fit_mean, fit_std, pred_mean, pred_std, title)

    return f1_train_mean, fit_mean, pred_mean

In [27]:
# splitting data 

X_train, X_test, y_train, y_test = split_data(df=df_churn_scaled,feature_ls=predictors_list, y_target='Churn',training= True)

(5634, 16) (1409, 16)


# Random hill experiment 

In [ ]:
# testing random hill using 3 restarts and larning rate for 60X 60 NN 

In [43]:
import gc 
gc.collect()

31756

In [39]:
import mlrose_hiive as mlrose

In [42]:
%%time
f1_grid_search_rh = partial(f1_score, average="weighted")
param_list_rh = ({
  'learning_rate': [0.01, 0.1],
  'restarts': [5, 15, 40],
  'activation': [mlrose.relu]
})
model_random_hill_nn = mlrose_hiive.NNGSRunner(X_train, y_train, 
                                 X_test, y_test, 
                                 "NN_RH",
                                 output_directory="./",
                                 seed=10, iteration_list=[10000], 
                                 algorithm=mlrose.random_hill_climb,
                                 hidden_layer_sizes=[[60,60]],
                                 grid_search_parameters=param_list_rh,
                                 grid_search_scorer_method=f1_grid_search_rh,
                                 cv=3)
#running search `
model_search_list_rh = model_random_hill_nn.run()

Running nngs_rhc
Fitting 3 folds for each of 6 candidates, totalling 18 fits
*****************
*** Run START ***
*****************
max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[0.02], fitness:[2.7636]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], c

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[1]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[2.70], fitness:[2.6300]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[2]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[3.10], fitness

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[3]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[5.09], fitness:[2.8087]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[3]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[2], iteration:[100], done:[True], time:[6.28], fitness

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[4]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[8.49], fitness:[2.5890]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[5]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[9.48], fitne

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[12]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[30.68], fitness:[2.7636]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[12]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[2], iteration:[100], done:[True], time:[31.96], f

...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[4]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[10.49], fitness:[2.5930]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[5]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[11.42], fit

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[12]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[34.37], fitness:[2.7697]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[12]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[2], iteration:[100], done:[True], time:[36.09], f

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[19]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[2], iteration:[100], done:[True], time:[68.28], fitness:[2.6241]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[20]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[71.85], f

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[27]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[113.07], fitness:[2.7697]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[27]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[114.78],

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[34]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[3], iteration:[100], done:[True], time:[164.73], fitness:[2.7397]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[35]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[170.84],

In [46]:
model_search_list_rh[2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_restarts,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,11.317145,0.213240,0.007766,0.000258,relu,"[60, 60]",0.01,5,{'activation': <function relu at 0x7f91f6a8fea...,0.621359,0.612906,0.627422,0.620562,0.005953,3,0.620163,0.624131,0.617396,0.620563,0.002764
1,44.546118,1.256485,0.007258,0.000101,relu,"[60, 60]",0.01,15,{'activation': <function relu at 0x7f91f6a8fea...,0.622368,0.611454,0.625960,0.619928,0.006168,5,0.618707,0.624154,0.616924,0.619928,0.003075
2,199.147084,0.535444,0.007271,0.000170,relu,"[60, 60]",0.01,40,{'activation': <function relu at 0x7f91f6a8fea...,0.621848,0.613385,0.630912,0.622048,0.007157,1,0.622651,0.626862,0.618131,0.622548,0.003565
3,10.804901,0.204050,0.007182,0.000342,relu,"[60, 60]",0.10,5,{'activation': <function relu at 0x7f91f6a8fea...,0.621359,0.612906,0.627422,0.620562,0.005953,3,0.620163,0.624131,0.617396,0.620563,0.002764
4,42.530237,0.084738,0.010022,0.004068,relu,"[60, 60]",0.10,15,{'activation': <function relu at 0x7f91f6a8fea...,0.622368,0.611454,0.625960,0.619928,0.006168,5,0.618707,0.624154,0.616924,0.619928,0.003075
5,541.866741,484.165963,0.007796,0.000145,relu,"[60, 60]",0.10,40,{'activation': <function relu at 0x7f91f6a8fea...,0.621848,0.613385,0.630912,0.622048,0.007157,1,0.622651,0.626862,0.618131,0.622548,0.003565


In [49]:
# predicting using best fit 

In [48]:
y_pred = model_search_list_rh[3].predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.60      0.70      1058
           1       0.34      0.61      0.44       351

    accuracy                           0.60      1409
   macro avg       0.58      0.61      0.57      1409
weighted avg       0.70      0.60      0.63      1409



In [ ]:
# ploting learning curve and time 
model_rh_train,model_rh_fit_time, model_rh_nn_pred_time = learning_curve(model_search_list_rh[3], X_train, y_train, title= "Neural_network with RH")

Fitting 3 folds for each of 6 candidates, totalling 18 fits
*****************
*** Run START ***
*****************
max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[0.00], fitness:[3.3395]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[2]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[15.45], fitness:[2.5345]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[2]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[3], iteration:[100], done:[True], time:[15.59], fitne

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[4]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[31.87], fitness:[2.7215]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[4]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[32.03], fitne

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[6]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[48.63], fitness:[3.3395]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[6]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[48.79], fit

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[14]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[123.12], fitness:[3.3395]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[15], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[14]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[4], iteration:[100], done:[True], time:[123.29],

...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[30]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[304.84], fitness:[3.3395]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[30]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[305.01],

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[38]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[414.53], fitness:[3.3395]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[38]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[414.71],

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[5]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[40.74], fitness:[2.5345]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[5]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[2], iteration:[100], done:[True], time:[40.89], fit

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[13]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[115.45], fitness:[2.5345]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[13]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[2], iteration:[100], done:[True], time:[115.61],

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[21]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[0], iteration:[0], done:[False], time:[201.23], fitness:[2.5345]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102// -0.3802517 ]...

max_iters:[100], learning_rate:[0.01], restarts:[40], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.03739392 -0.84258102
 -0.3802517 ]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[21]
runner_name:[nngs_rhc], experiment_name:[NN_RH], attempt:[1], iteration:[100], done:[True], time:[201.38],

In [56]:
# simulated annealing 

In [45]:
from mlrose_hiive import ExpDecay
f1_grid_search_sa = partial(f1_score, average="weighted")
param_list_sa = ({
  'schedule': [ExpDecay(1),ExpDecay(10),ExpDecay(25),ExpDecay(50), ExpDecay(100),ExpDecay(10000)],
  'learning_rate': [0.01, 0.1, 0.05],  
  'activation': [mlrose.relu],
  'max_iters': [10000]
})

# running search 
model_simulated_annealing_nn  = mlrose_hiive.NNGSRunner(x_train=X_train,y_train=y_train,
                     x_test=X_test,
                     y_test=y_test,
                     experiment_name='SA_NN_hp_search',
                     output_directory="./", seed = 1,
                     hidden_layer_sizes=[[60,60]],algorithm=mlrose.simulated_annealing,
                      grid_search_parameters=param_list_sa,iteration_list=[10000],
                                 grid_search_scorer_method=f1_grid_search_sa
                     )


model_search_list_sa = model_simulated_annealing_nn.run()   

Running nngs_sa
Fitting 5 folds for each of 18 candidates, totalling 90 fits
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[1], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.02], fitness:[3.5733]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[1], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[10], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.5893]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[10], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[25], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.6444]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[25], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[100], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.5818]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[100], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_nam

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[10000], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.6749]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[10000], learning_rate:[0.01], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment

...

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[1], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.6749]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[1], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[S

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[25], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.5733]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[25], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[50], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.5893]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[50], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:

***************
*** Run END ***
***************
*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[100], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[SA_NN_hp_search], attempt:[0], iteration:[0], done:[False], time:[0.01], fitness:[3.6444]
	[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717//  0.89793628]...

max_iters:[10000], schedule:[100], learning_rate:[0.05], init_state:[[-0.16595599  0.44064899 -0.99977125 ...  0.87960362 -0.33935717
  0.89793628]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_nam

***************
*** Run END ***
***************
Run time: 18945.923189521
Saving: [./SA_NN_hp_search/nngs_sa__SA_NN_hp_search__run_stats_df__7D7735A1564B4EAFC89621C22730CE3B.csv]
Saving: [./SA_NN_hp_search/nngs_sa__SA_NN_hp_search__curves_df__7D7735A1564B4EAFC89621C22730CE3B.csv]
Saving: [./SA_NN_hp_search/nngs_sa__SA_NN_hp_search__cv_results_df__7D7735A1564B4EAFC89621C22730CE3B.csv]
Saving: [./SA_NN_hp_search/nngs_sa__SA_NN_hp_search__grid_search_results__7D7735A1564B4EAFC89621C22730CE3B.p]


In [50]:
y_pred = model_search_list_sa[3].predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86      1058
           1       0.56      0.50      0.53       351

    accuracy                           0.78      1409
   macro avg       0.70      0.68      0.69      1409
weighted avg       0.77      0.78      0.77      1409



In [53]:
model_search_list_sa[2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_max_iters,param_schedule,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,205.017880,2.275897,0.005183,0.000062,relu,"[60, 60]",0.01,10000,1,{'activation': <function relu at 0x7f91f6a8fea...,0.777417,0.766959,0.766260,0.795999,0.784677,0.778262,0.011210,1,0.781224,0.782628,0.783070,0.774057,0.774569,0.779110,0.003967
1,203.480190,1.905437,0.005004,0.000412,relu,"[60, 60]",0.01,10000,10,{'activation': <function relu at 0x7f91f6a8fea...,0.774460,0.768564,0.753817,0.796502,0.781072,0.774883,0.014068,4,0.775969,0.783762,0.785959,0.771502,0.776963,0.778831,0.005301
2,204.762041,0.251197,0.004953,0.000182,relu,"[60, 60]",0.01,10000,25,{'activation': <function relu at 0x7f91f6a8fea...,0.772943,0.777767,0.750368,0.781704,0.780802,0.772717,0.011586,7,0.775374,0.774213,0.777700,0.766559,0.774444,0.773658,0.003758
3,205.107853,0.415311,0.005283,0.000551,relu,"[60, 60]",0.01,10000,50,{'activation': <function relu at 0x7f91f6a8fea...,0.772528,0.764491,0.750982,0.791251,0.778210,0.771492,0.013463,10,0.775109,0.781434,0.775329,0.768566,0.775612,0.775210,0.004076
4,204.572508,0.229605,0.004915,0.000163,relu,"[60, 60]",0.01,10000,100,{'activation': <function relu at 0x7f91f6a8fea...,0.778180,0.767803,0.762187,0.780407,0.766886,0.771093,0.006997,13,0.780321,0.784405,0.784818,0.766863,0.777839,0.778849,0.006532
5,205.761231,0.369213,0.005316,0.000478,relu,"[60, 60]",0.01,10000,10000,{'activation': <function relu at 0x7f91f6a8fea...,0.776150,0.764093,0.739078,0.767150,0.778939,0.765082,0.014114,16,0.773651,0.782948,0.774736,0.762626,0.775921,0.773976,0.006539
6,206.000589,4.043277,0.005346,0.000285,relu,"[60, 60]",0.10,10000,1,{'activation': <function relu at 0x7f91f6a8fea...,0.777417,0.766959,0.766260,0.795999,0.784677,0.778262,0.011210,1,0.781224,0.782628,0.783070,0.774057,0.774569,0.779110,0.003967
7,204.993025,0.725298,0.004819,0.000133,relu,"[60, 60]",0.10,10000,10,{'activation': <function relu at 0x7f91f6a8fea...,0.774460,0.768564,0.753817,0.796502,0.781072,0.774883,0.014068,4,0.775969,0.783762,0.785959,0.771502,0.776963,0.778831,0.005301
8,212.847699,2.513471,0.005077,0.000335,relu,"[60, 60]",0.10,10000,25,{'activation': <function relu at 0x7f91f6a8fea...,0.772943,0.777767,0.750368,0.781704,0.780802,0.772717,0.011586,7,0.775374,0.774213,0.777700,0.766559,0.774444,0.773658,0.003758
9,211.430353,1.790758,0.004909,0.000202,relu,"[60, 60]",0.10,10000,50,{'activation': <function relu at 0x7f91f6a8fea...,0.772528,0.764491,0.750982,0.791251,0.778210,0.771492,0.013463,10,0.775109,0.781434,0.775329,0.768566,0.775612,0.775210,0.004076


In [54]:
model_search_list_sa

(   Iteration   Fitness        Time  \
 0          0  3.612746    0.017945   
 1      10000  0.490158  215.703645   
 
                                                State schedule_type  \
 0  [-0.165955990594852, 0.4406489868843162, -0.99...   exponential   
 1  [-0.46595599059485193, 0.6406489868843162, -0....   exponential   
 
    schedule_init_temp  schedule_exp_const  schedule_min_temp  \
 0                   1               0.005              0.001   
 1                   1               0.005              0.001   
 
    schedule_current_value  max_iters schedule  learning_rate  \
 0                0.999910      10000        1           0.01   
 1                0.340099      10000        1           0.01   
 
                                           init_state algorithm activation  \
 0  [-0.165955990594852, 0.4406489868843162, -0.99...        sa       relu   
 1  [-0.165955990594852, 0.4406489868843162, -0.99...        sa       relu   
 
    bias  early_stopping      clip_m